In [1]:
import numpy as np
from random import randint
from itertools import combinations

# Задание 1

In [118]:
def comb(a, b):
    n, m = len(a), len(b)
    t = 0
    for ind in combinations([i for i in range(n + m)], n):
        s = ''
        i, j = 0, 0
        for c in range(n + m):
            if c in ind:
                s += a[i]
                i += 1
            else:
                s += b[j]
                j += 1
        t += 1
        print(t, ') ', s, ':', sep='', end='')
        
        a1, b1 = list('-'*(n+m)), list('-'*(n+m))
        if 0 in ind:
            a1[0] = s[0]
            tr = 0
        else:
            b1[0] = s[0]
            tr = 1
        c = 1
        while c < n + m:
            if c in ind and tr == 0:
                a1[c] = s[c]
            elif c in ind and tr != 0:
                a1[c-1] = s[c]
                tr = 0
            elif c not in ind and tr == 0:
                b1[c] = s[c]
                tr = 1
            elif c not in ind and tr != 0:
                b1[c] = s[c]
            c += 1
            
        for i in range(n + m)[::-1]:
            if a1[i] == '-' and b1[i] == '-':
                del a1[i]
                del b1[i]
                
        print('\t', ''.join(a1))
        print(' '*(n+m+3), '\t', ''.join(b1), '\n')

Процесс конструирования выравниваний строк длин $ n $ и $ m $ сводится к размещению символов одной строки среди символов другой без изменения порядка следования символов каждой из строк между собой. В данном случае несколько подряд идущих символов одной строки соответствуют разрыву соответствующего размера в другой. Таких комбинаций сохраняющих порядок внутри каждой из строк существует $ C^{n}_{n+m} = C^{m}_{n+m} $

Рассмотрим пример. Найдем все выравнивания строк ABCD и xyz и подсчитаем иx количество. Приведенный алгоритм предсказывает $ C^{3}_{7} = 35 $ выравниваний.

Запустим функцию, генерирующую такие выравнивания и убедимся в наличии всех возможных вариантов!

In [119]:
comb('ABCD', 'efj')

1) ABCDefj:	 ABCD---
           	 ----efj 

2) ABCeDfj:	 ABCD--
           	 ---efj 

3) ABCefDj:	 ABC-D-
           	 ---efj 

4) ABCefjD:	 ABC--D
           	 ---efj 

5) ABeCDfj:	 ABCD--
           	 --e-fj 

6) ABeCfDj:	 ABCD-
           	 --efj 

7) ABeCfjD:	 ABC-D
           	 --efj 

8) ABefCDj:	 AB-CD-
           	 --ef-j 

9) ABefCjD:	 AB-CD
           	 --efj 

10) ABefjCD:	 AB--CD
           	 --efj- 

11) AeBCDfj:	 ABCD--
           	 -e--fj 

12) AeBCfDj:	 ABCD-
           	 -e-fj 

13) AeBCfjD:	 ABC-D
           	 -e-fj 

14) AeBfCDj:	 ABCD-
           	 -ef-j 

15) AeBfCjD:	 ABCD
           	 -efj 

16) AeBfjCD:	 AB-CD
           	 -efj- 

17) AefBCDj:	 A-BCD-
           	 -ef--j 

18) AefBCjD:	 A-BCD
           	 -ef-j 

19) AefBjCD:	 A-BCD
           	 -efj- 

20) AefjBCD:	 A--BCD
           	 -efj-- 

21) eABCDfj:	 ABCD--
           	 e---fj 

22) eABCfDj:	 ABCD-
           	 e--fj 

23) eABCfjD:	 ABC-D
           	 e--fj 

24) eABfCDj:	 ABCD-
           	 e-f-j 

25)

# Задание 2

Можно считать строки из файла

In [ ]:
# a, b = '', ''

# with open('strings_for_alignment.txt', 'r') as f:
#     a = f.readline().rstrip()
#     b = f.readline().rstrip()

А можно ввести вручную

In [140]:
a = 'editing'
b = 'distance'

Задаем массив весов

In [141]:
W = {'match':1, 'mismatch':-1, 'gap':-1}

## Выравниваем 

In [153]:
def align(a, b, W):
    
    def diff(i, j):
        if a[i] == b[j]:
            return 'match'
        return 'mismatch'
    
    n, m = len(a), len(b)

    D = np.empty((n+1, m+1))
    D[0,:] = np.arange(m+1)*W['gap']
    D[:,0] = np.arange(n+1)*W['gap']
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            D[i, j] = max(D[i-1, j] + W['gap'], D[i, j-1] + W['gap'], D[i-1, j-1] + W[diff(i-1, j-1)])
            
    i, j = n, m
    a1, b1, mtc = '', '', ''

    while i + j > 0:
        if D[i, j] == D[i-1, j] + W['gap']:
            a1 += a[i-1]
            b1 += '-'
            mtc += ' '
            i -= 1
        elif D[i, j] == D[i, j-1] + W['gap']:
            a1 += '-'
            b1 += b[j-1]
            mtc += ' '
            j -= 1
        else:
            a1 += a[i-1]
            b1 += b[j-1]
            if a[i-1] == b[j-1]:
                mtc += '|'
            else:
                mtc += ' '
            i -= 1
            j -= 1
            
    print(a1[::-1])
    print(mtc[::-1])
    print(b1[::-1])

In [147]:
align(a, b, W)

edi-ting-
 || | |  
-distance


### Тест 1

In [149]:
a = 'qqqaaaaaGaaaaa'
b = 'aaaaaCaaaaabbb'

In [150]:
align(a, b, W)

qqqaaaaaGaaaaa---
   ||||| |||||   
---aaaaaCaaaaabbb


### Тест 2

In [154]:
W = {'match':1, 'mismatch':-1, 'gap':-0.499}

In [155]:
align(a, b, W)

qqqaaaaa-Gaaaaa---
   |||||  |||||   
---aaaaaC-aaaaabbb


# Задание 3

Считывем из файла таблицу весов PAM250

In [138]:
W = np.empty((25,25), dtype=np.int)
ind = {}

f = open('PAM_250.txt', 'r')
amino = f.readline().split()[1:]
ind = {amino[i]:i for i in range(25)}
for i in range(25):
    W[i,:] = np.array(f.readline().split()[2:])
    
f.close()

Создаем независимо мутировавшие аминокислотные последовательности 

In [122]:
def mutogenesis(s, num, max_len):
    for _ in range(num):
        pos = randint(0, len(s)-1)
        ins = randint(1,max_len)
        dlt = randint(1,max_len)
        s = s[:pos] + ''.join([amino[randint(0, 19)] for _ in range(ins)]) + s[pos+dlt:]
    return s

In [124]:
A = ''.join([amino[randint(0, 19)] for _ in range(70)])
B = A
A = mutogenesis(A, 10, 3)
B = mutogenesis(B, 10, 3)
print(A)
print(B)

KTMYRYIMTNYAKANNSYWWLRWHTGFKIYNEMDGEYCYLSCMDIHWQDWLMYFMPGAQCLEHDGCDQYTFRT
KTFYFRYIMTYICKGLSYAMIDGQYNEMDGEYCYLSCMDIHWQDYYLIRCMPGAQCMCDAPHDGCPGGTM


Понравились? Запомним их

In [125]:
a, b = A, B

## Выравниваем

In [136]:
def align_w(a, b, W): 
    
    def gap_w(s):
        w = np.zeros((len(s)+1))
        for i in range(1, len(s)+1):
            w[i] = w[i-1] + W[ind[s[i-1]], ind['-']]
        return w
    
    def edit_dist(a, b):
        w = 0
        for i in range(len(a)):
            w += W[ind[a[i]], ind[b[i]]]
        return w

    n, m = len(a), len(b)
    D = np.empty((n+1, m+1), dtype=np.int)
    D[:,0] = gap_w(a)
    D[0,:] = gap_w(b)
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            a_gap = D[i, j-1] + W[ind[b[j-1]], ind['-']]
            b_gap = D[i-1, j] + W[ind[a[i-1]], ind['-']]
            match = D[i-1, j-1] + W[ind[a[i-1]], ind[b[j-1]]]
            D[i, j] = max(a_gap, b_gap, match)
            
    i, j = n, m
    a1, b1, mtc = '', '', ''

    while i + j > 0:
        if D[i, j] == D[i-1, j] + W[ind[a[i-1]], ind['-']]:
            a1 += a[i-1]
            b1 += '-'
            mtc += ' '
            i -= 1
        elif D[i, j] == D[i, j-1] + W[ind[b[j-1]], ind['-']]:
            a1 += '-'
            b1 += b[j-1]
            mtc += ' '
            j -= 1
        else:
            a1 += a[i-1]
            b1 += b[j-1]
            if a[i-1] == b[j-1]:
                mtc += '|'
            else:
                mtc += ' '
            i -= 1
            j -= 1
            
    assert edit_dist(a1,b1) == D[-1,-1]
    
    print(a1[::-1])
    print(mtc[::-1])
    print(b1[::-1])
    
    return a1, b1

In [139]:
a1, b1 = align_w(a, b, W)
print()

KTMY-RYIMTNYAKANNSYWWLRWHTGFKIYNEMDGEYCYLSCMDIHWQDW-LMYFMPGAQCL-E--HDGCDQYTFRT
|| | ||||| |     ||       |   ||||||||||||||||||||  |   ||||||     ||||   |   
KTFYFRYIMT-YICKGLSYA-MI-D-G-Q-YNEMDGEYCYLSCMDIHWQDYYLIRCMPGAQCMCDAPHDGCPGGTM--



### Test 

Попробуем испортить как минимум последнюю позицию выравнивания

In [134]:
W[ind[a1[0]], ind[b1[0]]] = -9999
W[ind[b1[0]], ind[a1[0]]] = -9999

In [135]:
align_w(a, b, W)
print()

KTMY-RYIMTNYAKANNSYWWLRWHTGFKIYNEMDGEYCYLSCMDIHWQDW-LMYFMPGAQCL-E--HDGCDQYTFRT
|| | ||||| |     ||       |   ||||||||||||||||||||  |   ||||||     ||||       
KTFYFRYIMT-YICKGLSYA-MI--DG-Q-YNEMDGEYCYLSCMDIHWQDYYLIRCMPGAQCMCDAPHDGCPG-G-TM



Готово!